### Name: Harsh Naik
### PRN: 013

In [1]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-

# Deep Neural Networks 
## Session 15b:

## Sequential Model using ionosphere putting it together

1. Monitor validation set as well as training set
2. L2 Regularization
3. Dropouts
4. Batch Norm
5. Early stopping

</p><img src='../../images/prasami_color_tutorials_small.png' style = 'width:400px;' alt="By Pramod Sharma : pramod.sharma@prasami.com" align="left"/>

In [2]:
###-----------------
### Import Libraries
###-----------------

import os
import sys
import numpy as np
import pandas as pd

from time import time
from collections.abc import Callable
from typing import Literal

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, ConfusionMatrixDisplay
from sklearn.preprocessing import StandardScaler, LabelEncoder
import matplotlib.pyplot as plt

import tensorflow as tf

%matplotlib inline

In [3]:
physical_devices = tf.config.list_physical_devices('GPU') 

if len(physical_devices) > 0:
    tf.config.experimental.set_memory_growth(physical_devices[0], True)

In [4]:
###----------------------
### Some basic parameters
###----------------------

inpDir = 'input' # location where input data is stored
outDir = '../output' # location to store outputs
modelDir = '../models'
subDir = 'fashion_mnist'

RANDOM_STATE = 24 # for initialization ----- REMEMBER: to remove at the time of promotion to production
np.random.seed(RANDOM_STATE) # Set Random Seed for reproducible  results
tf.random.set_seed(RANDOM_STATE) # setting for Tensorflow as well


EPOCHS = 100 # number of epochs
ALPHA = 0.1 # learning rate

BATCH_SIZE = 32 # other sizes leave last batch partial

PATIENCE = 7
# Set parameters for decoration of plots
params = {'legend.fontsize' : 'large',
          'figure.figsize'  : (15,15),
          'axes.labelsize'  : 'x-large',
          'axes.titlesize'  :'x-large',
          'xtick.labelsize' :'large',
          'ytick.labelsize' :'large',
         }

CMAP = plt.cm.coolwarm

plt.rcParams.update(params) # update rcParams

## Helper Functions

In [5]:
def fn_verify_dir(_path : str):
    '''
    Arg:
        path: path to verify the directory
    returns:
        create dir if it does not exists
    '''
    if os.path.exists(_path): # check if the path exists. Maybe a file or a folder
        
        print(_path, ' exists') # advised the user
        
    else:
        
        os.makedirs(_path) # create the path
        
        print("Created folder : ", _path)

In [6]:
###-----------------------------------
### Function to plot Loss Curve
###-----------------------------------

def plot_tf_hist(hist_df):
    '''
    Args:
      hist_df : pandas Dataframe with four columns
                For 'x' values, we will use index
    '''
    fig, axes = plt.subplots(1,2 , figsize = (15,6))

    # properties  matplotlib.patch.Patch 
    props = dict(boxstyle='round', facecolor='aqua', alpha=0.4)
    facecolor = 'cyan'
    fontsize=12
    
    # Get columns by index to eliminate any column naming error
    y1 = hist_df.columns[0]
    y2 = hist_df.columns[1]
    y3 = hist_df.columns[2]
    y4 = hist_df.columns[3]

    # Where was min loss
    best = hist_df[hist_df[y3] == hist_df[y3].min()]
    
    ax = axes[0]

    hist_df.plot(y = [y1,y3], ax = ax, colormap=CMAP)


    # little beautification
    txtFmt = "Loss: \n  train: {:6.4f}\n   test: {:6.4f}"
    txtstr = txtFmt.format(hist_df.iloc[-1][y1],
                           hist_df.iloc[-1][y3]) #text to plot
    
    # place a text box in upper middle in axes coords
    ax.text(0.3, 0.95, txtstr, transform=ax.transAxes, fontsize=fontsize,
            verticalalignment='top', bbox=props)

    # Mark arrow at lowest
    ax.annotate(f'Min: {best[y3].to_numpy()[0]:6.4f}', # text to print
                xy=(best.index.to_numpy(), best[y3].to_numpy()[0]), # Arrow start
                xytext=(best.index.to_numpy()-1, best[y3].to_numpy()[0]), # location of text 
                fontsize=fontsize, va='bottom', ha='right',bbox=props, # beautification of text
                arrowprops=dict(facecolor=facecolor, shrink=0.05)) # arrow

    # Draw vertical line at best value
    ax.axvline(x = best.index.to_numpy(), color = 'green', linestyle='-.', lw = 3);

    ax.set_xlabel("Epochs")
    ax.set_ylabel(y1.capitalize())
    ax.set_title('Errors')
    ax.grid();
    ax.legend(loc = 'upper left') # model legend to upper left

    ax = axes[1]

    hist_df.plot( y = [y2, y4], ax = ax, colormap=CMAP)
    
    # little beautification
    txtFmt = "Accuracy: \n  train: {:6.4f}\n  test:  {:6.4f}"
    txtstr = txtFmt.format(hist_df.iloc[-1][y2],
                           hist_df.iloc[-1][y4]) #text to plot

    # place a text box in upper middle in axes coords
    ax.text(0.3, 0.2, txtstr, transform=ax.transAxes, fontsize=fontsize,
            verticalalignment='top', bbox=props)

    # Mark arrow at lowest
    ax.annotate(f'Best: {best[y4].to_numpy()[0]:6.4f}', # text to print
                xy=(best.index.to_numpy(), best[y4].to_numpy()[0]), # Arrow start
                xytext=(best.index.to_numpy()-1, best[y4].to_numpy()[0]), # location of text 
                fontsize=fontsize, va='bottom', ha='right',bbox=props, # beautification of text
                arrowprops=dict(facecolor=facecolor, shrink=0.05)) # arrow
    
    
    # Draw vertical line at best value
    ax.axvline(x = best.index.to_numpy(), color = 'green', linestyle='-.', lw = 3);

    ax.set_xlabel("Epochs")
    ax.set_ylabel(y2.capitalize())
    ax.grid()
    ax.legend(loc = 'lower left')
    
    plt.tight_layout()

## Load Data

### Data Set Information:

This radar data was collected by a system in Goose Bay, Labrador. This system consists of a phased array of 16 high-frequency antennas with a total transmitted power on the order of 6.4 kilowatts. See the paper for more details. The targets were free electrons in the ionosphere. "Good" radar returns are those showing evidence of some type of structure in the ionosphere. "Bad" returns are those that do not; their signals pass through the ionosphere.

Received signals were processed using an autocorrelation function whose arguments are the time of a pulse and the pulse number. There were 17 pulse numbers for the Goose Bay system. Instances in this database are described by 2 attributes per pulse number, corresponding to the complex values returned by the function resulting from the complex electromagnetic signal.

Attribute Information:

-- All 34 are continuous
-- The 35th attribute is either "good" or "bad" according to the definition summarized above. This is a binary classification task.

We will model this as two class.

In [7]:
train_df = pd.read_csv(os.path.join(inpDir, subDir, 'fashion-mnist_train.csv'))
test_df = pd.read_csv(os.path.join(inpDir, subDir, 'fashion-mnist_test.csv'))

train_df.shape,test_df.shape

FileNotFoundError: [Errno 2] No such file or directory: 'input\\fashion_mnist\\fashion-mnist_train.csv'

In [ ]:
train_df.head()

In [ ]:
train_df["label"]

In [ ]:
X_train=train_df.drop('label', axis = 1).values
y_train= train_df[["label"]]

X_test=test_df.drop('label', axis = 1).values
y_test= test_df[["label"]]

In [ ]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

le = LabelEncoder()

y_train = le.fit_transform(y_train)

y_test = le.transform(y_test)

In [ ]:
sc = StandardScaler()

X_train = sc.fit_transform(X_train)

X_test = sc.transform(X_test)

In [ ]:
initalizer = tf.keras.initializers.GlorotUniform(seed = RANDOM_STATE)

In [ ]:
X_train.shape

## Vanilla Model

In [ ]:
model1 = tf.keras.models.Sequential([
    
    tf.keras.layers.Dense (784,
                           activation = tf.keras.activations.relu,
                           kernel_initializer = initalizer),
    
    tf.keras.layers.Dense (512,
                           activation = tf.keras.activations.relu,
                           kernel_initializer = initalizer),
    
    tf.keras.layers.Dense (256,
                           activation = tf.keras.activations.relu,
                           kernel_initializer = initalizer),
    
    tf.keras.layers.Dense (128,
                           activation = tf.keras.activations.relu,
                           kernel_initializer = initalizer),
    
    tf.keras.layers.Dense (64,
                           activation = tf.keras.activations.relu,
                           kernel_initializer = initalizer),
    
    tf.keras.layers.Dense (32,
                           activation = tf.keras.activations.relu,
                           kernel_initializer = initalizer),
    
    tf.keras.layers.Dense (10),
    
])

In [ ]:
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy( from_logits = True)

In [ ]:
model1.compile (optimizer = 'Adam',
               loss = loss_fn,
               metrics= ['accuracy'])

In [ ]:
checkpoint_filepath = os.path.join(modelDir, subDir, 'weights_all_in')

model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=True,
    monitor = 'val_loss',
    mode = 'auto',
    save_best_only = True
)


early_stopping_callback = tf.keras.callbacks.EarlyStopping(
    monitor = 'val_loss',
    patience=PATIENCE,
    mode='auto',
    baseline =None,
    restore_best_weights=True)


In [ ]:
histroy = model1.fit(x = X_train, y= y_train,
                     epochs=EPOCHS,
                     batch_size = BATCH_SIZE,
                     validation_data=[X_test, y_test],
                     verbose = 2,
                    callbacks = [early_stopping_callback, model_checkpoint_callback])

In [ ]:
model1.summary()

In [ ]:
hist_df = pd.DataFrame(histroy.history)
hist_df.head()

In [ ]:
plot_tf_hist(hist_df)

In [ ]:
hist_df[hist_df['val_loss'] == hist_df['val_loss'].min()]

## L2 Regularizer

In [ ]:
initalizer = tf.keras.initializers.GlorotUniform(seed = RANDOM_STATE)

regularizer = tf.keras.regularizers.L2(0.02)

model2 = tf.keras.models.Sequential([    
    
    
    
    tf.keras.layers.Dense (512,
                           activation = tf.keras.activations.relu,
                           kernel_initializer = initalizer,
                           kernel_regularizer = regularizer),
    
     tf.keras.layers.Dense (256,
                           activation = tf.keras.activations.relu,
                           kernel_initializer = initalizer,
                           kernel_regularizer = regularizer),
    
    tf.keras.layers.Dense (128,
                           activation = tf.keras.activations.relu,
                           kernel_initializer = initalizer,
                           kernel_regularizer = regularizer),
    
        tf.keras.layers.Dense (64,
                           activation = tf.keras.activations.relu,
                           kernel_initializer = initalizer,
                           kernel_regularizer = regularizer),
        
        tf.keras.layers.Dense (32,
                           activation = tf.keras.activations.relu,
                           kernel_initializer = initalizer,
                           kernel_regularizer = regularizer),
    
    tf.keras.layers.Dense (10),
    
])


loss_fn = tf.keras.losses.SparseCategoricalCrossentropy( from_logits = True)

model2.compile (optimizer = 'Adam',
               loss = loss_fn,
               metrics= ['accuracy'])


history = model2.fit( x = X_train, 
                     y= y_train,
                    epochs=EPOCHS,
                    batch_size = BATCH_SIZE,
                    validation_data=[X_test, y_test],
                    verbose = 2,
                    callbacks = [early_stopping_callback, model_checkpoint_callback])

hist_df = pd.DataFrame(history.history)

In [ ]:
plot_tf_hist(hist_df)

## Batch Normalization

In [ ]:
initalizer = tf.keras.initializers.GlorotUniform(seed = RANDOM_STATE)

model3 = tf.keras.models.Sequential([
    
    tf.keras.layers.Dense (784,
                           #activation = tf.keras.activations.relu,
                           kernel_initializer = initalizer
                           ),
    
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Activation(tf.keras.activations.relu),
    
    tf.keras.layers.Dense (512,
                           #activation = tf.keras.activations.relu,
                           kernel_initializer = initalizer),
    
    tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Activation(tf.keras.activations.relu),
    
    tf.keras.layers.Dense (256,
                           #activation = tf.keras.activations.relu,
                           kernel_initializer = initalizer),
    
    tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Activation(tf.keras.activations.relu),
        
    tf.keras.layers.Dense (128,
                           #activation = tf.keras.activations.relu,
                           kernel_initializer = initalizer),
    
    tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Activation(tf.keras.activations.relu),
            
    tf.keras.layers.Dense (64,
                           #activation = tf.keras.activations.relu,
                           kernel_initializer = initalizer),
    
    tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Activation(tf.keras.activations.relu),
    
    tf.keras.layers.Dense (10),
    
])


loss_fn = tf.keras.losses.SparseCategoricalCrossentropy( from_logits = True)

model3.compile (optimizer = 'Adam',
               loss = loss_fn,
               metrics= ['accuracy'])


history = model3.fit( x = X_train, 
                     y= y_train,
                    epochs=EPOCHS,
                    batch_size = BATCH_SIZE,
                    validation_data=[X_test, y_test],
                    verbose = 2,
                    callbacks = [early_stopping_callback, model_checkpoint_callback])

hist_df = pd.DataFrame(history.history)

In [ ]:
plot_tf_hist(hist_df)

## Dropouts

In [ ]:
initalizer = tf.keras.initializers.GlorotUniform(seed = RANDOM_STATE)

rate = 0.1

model4 = tf.keras.models.Sequential([
    
    tf.keras.layers.Dense (784,
                           activation = tf.keras.activations.relu,
                           kernel_initializer = initalizer
                           ),
    
    tf.keras.layers.Dropout(rate , seed = RANDOM_STATE),
    
    tf.keras.layers.Dense (512,
                           activation = tf.keras.activations.relu,
                           kernel_initializer = initalizer
                           ),
    
    tf.keras.layers.Dropout(rate , seed = RANDOM_STATE),
    
    tf.keras.layers.Dense (256,
                           activation = tf.keras.activations.relu,
                           kernel_initializer = initalizer),
    
    tf.keras.layers.Dropout(rate , seed = RANDOM_STATE),
    
    tf.keras.layers.Dense (128,
                           activation = tf.keras.activations.relu,
                           kernel_initializer = initalizer),
    
    tf.keras.layers.Dropout(rate , seed = RANDOM_STATE),
    tf.keras.layers.Dense (64,
                           activation = tf.keras.activations.relu,
                           kernel_initializer = initalizer
                           ),
    
    tf.keras.layers.Dropout(rate , seed = RANDOM_STATE),
    
    tf.keras.layers.Dense (10),
    
])


loss_fn = tf.keras.losses.SparseCategoricalCrossentropy( from_logits = True)

model4.compile (optimizer = 'Adam',
               loss = loss_fn,
               metrics= ['accuracy'])


history = model4.fit( x = X_train, 
                     y= y_train,
                    epochs=EPOCHS,
                    batch_size = BATCH_SIZE,
                    validation_data=[X_test, y_test],
                    verbose = 2,
                    callbacks = [early_stopping_callback, model_checkpoint_callback])

hist_df = pd.DataFrame(history.history)

In [ ]:
plot_tf_hist(hist_df)

### All In

In [ ]:
checkpoint_filepath = os.path.join(modelDir, subDir, 'weights_all_in')

model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=True,
    monitor = 'val_loss',
    mode = 'auto',
    save_best_only = True
)


early_stopping_callback = tf.keras.callbacks.EarlyStopping(
    monitor = 'val_loss',
    patience=PATIENCE,
    mode='auto',
    baseline =None,
    restore_best_weights=True)


In [ ]:
initalizer = tf.keras.initializers.GlorotUniform(seed = RANDOM_STATE)

rate = 0.05

model = tf.keras.models.Sequential([
    
    tf.keras.layers.InputLayer( input_shape =(784, ) ), #### 
    
    tf.keras.layers.Dense (784,
                           activation = tf.keras.activations.relu,
                           kernel_initializer = initalizer,
                           kernel_regularizer = regularizer),
    
    tf.keras.layers.BatchNormalization(),
    
    tf.keras.layers.Dropout(rate , seed = RANDOM_STATE),
    
    tf.keras.layers.Dense (512,
                           activation = tf.keras.activations.relu,
                           kernel_initializer = initalizer,
                           kernel_regularizer = regularizer),
    
    tf.keras.layers.BatchNormalization(),
    
    tf.keras.layers.Dropout(rate , seed = RANDOM_STATE),
    
    tf.keras.layers.Dense (256,
                           activation = tf.keras.activations.relu,
                           kernel_initializer = initalizer,
                           kernel_regularizer = regularizer),
    
    tf.keras.layers.BatchNormalization(),
    
    tf.keras.layers.Dropout(rate , seed = RANDOM_STATE),
    
    tf.keras.layers.Dense (128,
                           activation = tf.keras.activations.relu,
                           kernel_initializer = initalizer,
                           kernel_regularizer = regularizer),
    
    tf.keras.layers.BatchNormalization(),
    
    tf.keras.layers.Dropout(rate , seed = RANDOM_STATE),
    
    tf.keras.layers.Dense (64,
                           activation = tf.keras.activations.relu,
                           kernel_initializer = initalizer,
                           kernel_regularizer = regularizer),
    
    tf.keras.layers.BatchNormalization(),
    
    tf.keras.layers.Dropout(rate , seed = RANDOM_STATE),
    
    tf.keras.layers.Dense (10),
    
])


loss_fn = tf.keras.losses.SparseCategoricalCrossentropy( from_logits = True)

model.compile (optimizer = 'Adam',
               loss = loss_fn,
               metrics= ['accuracy'])

In [ ]:
history = model.fit( x = X_train, 
                     y= y_train,
                    epochs=EPOCHS,
                    batch_size = BATCH_SIZE,
                    validation_data=[X_test, y_test],
                    verbose = 2,
                    callbacks = [early_stopping_callback, model_checkpoint_callback])

hist_df = pd.DataFrame(history.history)

In [ ]:
plot_tf_hist(hist_df)

In [ ]:
loss_dict = model.evaluate(X_test, y_test, verbose=1 , return_dict=True)

In [ ]:
loss_dict

In [ ]:
hist_df[hist_df['val_loss'] == hist_df['val_loss'].min()]

In [ ]:
del model
del model1
del model2
del model3
del model4

In [ ]:
initalizer = tf.keras.initializers.GlorotUniform(seed = RANDOM_STATE)

rate = 0.05

model = tf.keras.models.Sequential([
    
    tf.keras.layers.InputLayer( input_shape =(784, ) ), #### 
    
    tf.keras.layers.Dense (784,
                           activation = tf.keras.activations.relu,
                           kernel_initializer = initalizer,
                           kernel_regularizer = regularizer),
    
    tf.keras.layers.BatchNormalization(),
    
    tf.keras.layers.Dropout(rate , seed = RANDOM_STATE),
    
    tf.keras.layers.Dense (512,
                           activation = tf.keras.activations.relu,
                           kernel_initializer = initalizer,
                           kernel_regularizer = regularizer),
    
    tf.keras.layers.BatchNormalization(),
    
    tf.keras.layers.Dropout(rate , seed = RANDOM_STATE),
    
    tf.keras.layers.Dense (256,
                           activation = tf.keras.activations.relu,
                           kernel_initializer = initalizer,
                           kernel_regularizer = regularizer),
    
    tf.keras.layers.BatchNormalization(),
    
    tf.keras.layers.Dropout(rate , seed = RANDOM_STATE),
    
        tf.keras.layers.Dense (128,
                           activation = tf.keras.activations.relu,
                           kernel_initializer = initalizer,
                           kernel_regularizer = regularizer),
    
    tf.keras.layers.BatchNormalization(),
    
    tf.keras.layers.Dropout(rate , seed = RANDOM_STATE),
    
        tf.keras.layers.Dense (64,
                           activation = tf.keras.activations.relu,
                           kernel_initializer = initalizer,
                           kernel_regularizer = regularizer),
    
    tf.keras.layers.BatchNormalization(),
    
    tf.keras.layers.Dropout(rate , seed = RANDOM_STATE),
    
    tf.keras.layers.Dense (10),
    
])


loss_fn = tf.keras.losses.SparseCategoricalCrossentropy( from_logits = True)

model.compile (optimizer = 'Adam',
               loss = loss_fn,
               metrics= ['accuracy'])

In [ ]:
res = model.evaluate(X_test, y_test, verbose=2, return_dict=True)

res['accuracy']

In [ ]:
model.load_weights(checkpoint_filepath)

In [ ]:
res = model.evaluate(X_test, y_test, verbose=2, return_dict=True)

res['accuracy']

## Save Complete Model

In [ ]:
model_path = os.path.join(modelDir, subDir, 'complete_all_in')

tf.keras.models.save_model(model, model_path)

In [ ]:
model1 = tf.keras.models.load_model(model_path)

In [ ]:
res = model1.evaluate(X_test, y_test, verbose=2, return_dict=True)

res['accuracy']

In [ ]:
model1.summary()